Similarly, if you need to use `Gurobi`

# 3. Thermodynamic variability analysis (TVA)

There exists multiple flux states that define the same optimum. TVA predicts the ranges of variables such as metabolic fluxes, Gibbs energy of reactions and metabolite concentrations by taking into account of thermodynamic uncertainities using multivariate approach.

As described previously, you can solve the TVA problem on either the MILP or MIQCP. Although, MILP is faster compared to solving MIQCP, MIQCP is thermodynamically consistent as they drawn from multivariate distribution. 

There are three different functions for running TVA in multiTFA. One for the MILP, the remaining two for MIQCP, depending on the solver you use (either Gurobi or Cplex).

By default, TVA is performed on all the variables in a model unless `variable_list` parameter is specified. It is a `list` of variable names you want to perform TVA on. For example reactons id's or names of Gibbs energy variables etc.

The example below demonstrates how to perform TVA on *E. coli* core model with constraints described in previous sections. We will use `cplex` for our calculations. In our observations `Cplex` ourperforms `Gurobi`. 


In [1]:
from multitfa.test_model import load_test_data
tfa_model = load_test_data() # Load the test model

from multitfa import analysis
dg_vars = [rxn.delG_forward.name for rxn in tfa_model.reactions if rxn.id not in tfa_model.Exclude_reactions] # Gibbs energy variable names


Fragments already downloaded
Fragments already downloaded
Fragments already downloaded
Using license file /home/vishnu/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /tmp/tmpgbxvilzh.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


Now let us perform TVA on `tfa_model` using univariate approch and calculate ranges on reaction flux variables

In [14]:
tfa_model.solver = 'cplex'
ranges_box = analysis.variability(tfa_model, variable_list = dg_vars) # TVA using MILP
print(ranges_box)

               minimum    maximum
dG_PFK      -65.527361  26.205939
dG_PFL      -72.197674  33.074152
dG_PGI      -19.085713  24.931203
dG_PGK        0.000000  59.219274
dG_PGL      -40.671665  28.293843
...                ...        ...
dG_ME2      -70.983610  57.688660
dG_NADH16  -194.698524 -61.742679
dG_NADTRHD  -58.319625  35.223606
dG_NH4t     -13.742207  32.066015
dG_PDH     -116.749219  38.743581

[72 rows x 2 columns]


 `ranges_box` is a `pandas DataFrame` which can be readily exported to a csv. 

In [ ]:
ranges_box.to_csv('ranges.csv')

Now lets solve the TVA with multivariate approach using MIQCP. There are two functions one when using `Gurobi` and another for `Cplex`.

In [ ]:
ranges_MIQC_cplex = analysis.variability_legacy_cplex(tfa_model,variable_list = dg_vars) # TVA using MIQCP with cplex
print(ranges_MIQC_cplex)

Setting the parameter `fraction_of_optim=0.9` would give variable ranges at 90% optimality of specified variable optimality. The variable should be specified by Setting `biomass_rxn` parameter.  

In [13]:
ranges_MIQC_cplex_mingrowth = analysis.variability_legacy_cplex(tfa_model,variable_list=dg_vars, biomass_rxn='BIOMASS_Ecoli_core_w_GAM', fraction_of_optim=0.9)
print(ranges_MIQC_cplex_mingrowth)


Selected objective sense:  MINIMIZE
Selected objective  name:  37118d97-5947-11eb-b03f-509a4c43d0a7
Selected RHS        name:  rhs
Selected bound      name:  bnd
                 minimum    maximum
dG_PFK     -6.349569e+01  24.174265
dG_PFL     -7.084883e+01  31.725309
dG_PGI     -1.940986e+01  25.255346
dG_PGK      8.314189e-09  56.021188
dG_PGL     -4.961171e+01  37.233887
...                  ...        ...
dG_ME2     -8.241713e+01  69.129418
dG_NADH16  -2.107758e+02 -33.736303
dG_NADTRHD -4.100270e+01  23.163226
dG_NH4t    -9.670293e+00  27.994102
dG_PDH     -1.200410e+02  31.809464

[72 rows x 2 columns]


Similarly, if you need to use `Gurobi`

In [ ]:
tfa_model.solver = 'gurobi'
ranges_MIQC_grb_mingrowth = analysis.variability_legacy_gurobi(tfa_model,variable_list=dg_vars, fraction_of_optim=0.9)
print(ranges_MIQC_grb_mingrowth)

Please note, runnig TVA on genome scale model is computationally expensive, especially using MIQC problems. We have observed that relaxing the `Gap` parameter helped achieving faster run times. 

## 3.1 Alternative way to solve MIQC

When `Gurobi` or `Cplex` is not available, and if you wanted to run MIQC problem, we developed a sampling based approach to solve the MIQCP. We sample the uncertainity variables on the surface of the ellipsoid and solve the subsequent MILP problem. This can be solved using MILP solvers. The exit criterion of the sampler can be chosen as either

(1) the number of samples since last improvement or 
(2) a fixed number of samples followed by use of a generalized extreme value distribution to infer the maximum value.

In [4]:
gev_ranges, gev_samples = analysis.gev_sampling(tfa_model, variable_list=dg_vars, cutoff = 100)

Read LP format model from file /tmp/tmpzx24mpn1.lp
Reading time = 0.01 seconds
: 504 rows, 622 columns, 2492 nonzeros


In [5]:
print(gev_samples)

            minimum    maximum       minimum    maximum     minimum  \
dG_PFK          0.0  20.189736 -5.811624e+01  17.212549  -56.265716   
dG_PFL          0.0  23.289971 -5.623179e+01  19.097002  -56.211262   
dG_PGI          0.0  22.840318 -1.607155e+01  21.592842  -15.594077   
dG_PGK          0.0  56.030152 -6.257324e-10  51.099571  -20.924373   
dG_PGL          0.0  26.877198 -3.448756e+01  22.009031  -34.640034   
...             ...        ...           ...        ...         ...   
dG_ME2          0.0  51.034598 -5.348991e+01  40.671081  -51.076494   
dG_NADH16       0.0 -50.438180 -1.571160e+02 -93.241204 -165.927104   
dG_NADTRHD      0.0  41.380232 -3.717106e+01  38.157733  -39.014512   
dG_NH4t         0.0  27.993973 -9.670293e+00  27.994102   -9.670293   
dG_PDH          0.0  33.986692 -8.756349e+01  24.936351  -91.428854   

               maximum     minimum    maximum       minimum    maximum  ...  \
dG_PFK       19.063074  -57.229705  18.099084 -5.676117e+01  18.5676

`gev_samples` is the alternate set of Gibbs energy of reactions and `gev_ranges` is the max-min ranges of Gibbs energy of reactions. By specifying `cutoff` parameter, we specify how many samples we use to train the extreme value distribution (default, 1000). The more samples the better. 

Alternatively, you could choose to use the other exit crieteria where if we don't see improvement since a defined number of samples.

In [7]:
cutoff_ranges, cutoff_samples, num_samples = analysis.cutoff_sampling(tfa_model, cutoff=10, variable_list=dg_vars)

Read LP format model from file /tmp/tmpk_dbj9vr.lp
Reading time = 0.01 seconds
: 504 rows, 622 columns, 2492 nonzeros


Here, `cutoff_ranges`, `cutoff_samples`, `num_samples` represents max-min Gibbs energy ranges of reactions, alternate Gibbs energy of reactions and num of samples to achieve the exit.

In [9]:
print(cutoff_samples)

            minimum    maximum       minimum    maximum       minimum  \
dG_PFK          0.0  19.969166 -5.817907e+01  17.149720 -5.708765e+01   
dG_PFL          0.0  23.019117 -5.699259e+01  18.336197 -5.723720e+01   
dG_PGI          0.0  22.796442 -1.543749e+01  22.226905 -1.602886e+01   
dG_PGK          0.0  55.244833  4.016329e-09  49.911694 -5.049515e-09   
dG_PGL          0.0  26.849433 -3.293232e+01  23.564273 -3.471207e+01   
...             ...        ...           ...        ...           ...   
dG_ME2          0.0  49.767854 -5.371313e+01  40.447855 -5.460166e+01   
dG_NADH16       0.0 -64.819093 -1.656771e+02 -90.348353 -1.494706e+02   
dG_NADTRHD      0.0  56.062960 -3.840931e+01  25.263307 -3.802311e+01   
dG_NH4t         0.0  27.994102 -9.670293e+00  27.994102 -9.670293e+00   
dG_PDH          0.0  33.986690 -8.988640e+01  23.106782 -9.254878e+01   

              maximum       minimum    maximum       minimum    maximum  ...  \
dG_PFK      18.241140 -5.709267e+01  18.236

## 3.2 Network embeded thermodynamics (NET)

By using the TVA apparoches demonstrated above, one can perform NET type of analysis. For example, we can fix the directionalities of all the reactions and sample for the metabolite concentrations. 

Reaction directionalities can be fixed by adjusting the lower or upper bounds of the reactions.

In [10]:
atps = tfa_model.reactions.get_by_id('ATPS4r')
print("lower bound is {} and upper bound is {}\n".format(atps.lower_bound, atps.upper_bound))
atps.lower_bound = 0
print("updated lower and upper bounds are {} \t {}\n".format(atps.lower_bound, atps.upper_bound))

lower bound is -1000.0 and upper bound is 1000.0

updated lower and upper bounds are 0 	 1000.0



We can access the metabolite concentration variables as shown below,

In [12]:
concentration_vars = [var.name for var in tfa_model.variables if var.name.startswith('lnc_')]
tfa_model.solver = 'cplex'
concentration_ranges = analysis.variability_legacy_cplex(tfa_model,variable_list = concentration_vars) # TVA using MIQCP with cplex
print(concentration_ranges)


Selected objective sense:  MINIMIZE
Selected objective  name:  37118d97-5947-11eb-b03f-509a4c43d0a7
Selected RHS        name:  rhs
Selected bound      name:  bnd

Selected objective sense:  MINIMIZE
Selected objective  name:  37118d97-5947-11eb-b03f-509a4c43d0a7
Selected RHS        name:  rhs
Selected bound      name:  bnd
                minimum   maximum
lnc_glc__D_e -11.512925 -3.912023
lnc_gln__L_c -11.512925 -3.912023
lnc_gln__L_e -11.512925 -3.912023
lnc_glu__L_c -11.512925 -3.912023
lnc_glu__L_e -11.512925 -3.912023
...                 ...       ...
lnc_fru_e    -11.512925 -3.912023
lnc_fum_c    -11.512925 -3.912023
lnc_fum_e    -11.512925 -3.912023
lnc_g3p_c    -11.512925 -3.912023
lnc_g6p_c    -11.512925 -3.912023

[72 rows x 2 columns]
